In this notebook we will export images from Polar front Sailbuoy data

In [1]:
import numpy as np
import win32com.client as win32
import glob
import os

# Select platform

Sermilik cruise 2023

In [14]:
# Location of data files
ev_template = 'D:/Sermilik/NNA_EV/Sermilik_dBDiff.EV'
n_files = 10

data_folder = 'D:/Sermilik/NNA_EK80/NNA_EK80/' 

processed_export_folder = 'D:/Sermilik/images_clean/' 
processed_export_resample_folder = 'D:/Sermilik/images_resampled/' 

cal_file='D:/Sermilik/NNA_EK80/NNA_EK80/Sermilik_20230806.ecs'

# Define variable name to export
#var_T1 = 'Calibration subset T1'
var_T2 = 'Calibration subset T2'
var_T3 = 'Calibration subset T3'
var_T4 = 'Calibration subset T4' #Passive
var_T5 = 'Calibration subset T5' #Passive
var_T6 = 'Calibration subset T6'
var_T7 = 'Calibration subset T7'#Passive

T_list = ['T2', 'T3', 'T4', 'T5', 'T6']

# Define variable name to export
#pvar_T1 = 'Processed data T1'  # 150 kHz
pvar_T2 = 'Processed data T2'   # 38 kHz
pvar_T3 = 'Processed data T3'   # 18 kHz
pvar_T4 = 'Processed data T4' #Passive  #70 kHz
pvar_T5 = 'Processed data T5' #Passive  #200 kHz 
pvar_T6 = 'Processed data T6'   # 120 kHz
#pvar_T7 = 'Processed data T7'#Passive  # 333 kHz


var_names = [var_T2,var_T3,var_T4,var_T5,var_T6]
processed_var_names = [pvar_T2,
                       pvar_T3,
                       pvar_T4, 
                       pvar_T5,
                       pvar_T6]
                       #pvar_T7]

# Find all data files with extension .raw for the date frame
date = '20230807'
raw_file_names = sorted(glob.glob(data_folder + f'NNA_*{date}*.raw'))
cal_file=f'D:/Sermilik/NNA_EK80/NNA_EK80/Sermilik_{date}.ecs'

In [15]:
raw_file_names

['D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T001004.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T002151.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T003407.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T004859.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T005819.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T010336.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T010828.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T011448.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T012146.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T012911.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T013007.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T013725.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T014441.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\NNA_Tarajoq-D20230807-T015128.raw',
 'D:/Sermilik/NNA_EK80/NNA_EK80\\N

# Export image


In [16]:
# Open new EV file

for ind in np.arange(0,len(raw_file_names)):
    # Connect to the Echoview application
    EvApp = win32.Dispatch("EchoviewCom.EvApplication") 
    EvApp.Minimize()

    # Create new file
    EvFile = EvApp.NewFile(ev_template)

    # Load data by chunks of 3 files (1 on period)
    Fileset = EvFile.Filesets.FindByName('Fileset 1')

    # Set cal file
    Fileset.SetCalibrationFile(cal_file)
    
    for sub_ind in range(0,n_files):
        Fileset.DataFiles.Add(raw_file_names[ind+sub_ind])
        
    image_file_name = os.path.basename(raw_file_names[ind])[:-4]
    image_file_resample_name = os.path.basename(raw_file_names[ind])[:-4]
    i=0
    for T_i in T_list:
        
        # Select variable and recalculate
        Processed_Var = EvFile.Variables.FindByName(f'Processed data {T_i}')
        Sv_Var = EvFile.Variables.FindByName(f'Attenuated signal removal {T_i}')
        
        # Create a new editable line that offsets and spans gaps from bottom line
        if i == 0:
            Offset_line = EvFile.Lines.FindByName('Bottom Smooth')
            New_Line = EvFile.Lines.CreateOffsetLinear(Offset_line, 1, -1, True) # Editable line with 2m offset from bottom, spans gaps
            
        
        Sv_Var.Properties.Analysis.ExcludeBelowLine = "Line 1" # Variable before processed data
        Processed_Var.Properties.Analysis.ExcludeBelowLine = "Line 1"

        Processed_Var.Properties.Grid.SetDepthRangeGrid(1,50)
        Processed_Var.Properties.Grid.SetTimeDistanceGrid(2,1)

        #Processed_Var.Properties(ColorScheme = "Viridis")
    
        Processed_Var.ExportEchogramToImage(f'{processed_export_folder}{image_file_name}_{T_i}.png',500,-1,-1)
        i+=1

        Resampled_Var = EvFile.Variables.FindByName(f'Resample by number of pings {T_i}')
        Resampled_Var.ExportEchogramToImage(f'{processed_export_resample_folder}{image_file_resample_name}_{T_i}.png',500,-1,-1)

        EvFile = EvApp.Save(f'{ev_template}_{date}')
        
    # Close file
    EvFile.Close()
    # Quit Echoview
    EvApp.Quit()

KeyboardInterrupt: 

In [11]:
            EvApp = win32.Dispatch("EchoviewCom.EvApplication") 
            EvApp.Minimize()
        
            # Create new file
            EvFile = EvApp.NewFile(ev_template)
        
            # Load data by chunks of 3 files (1 on period)
            Fileset = EvFile.Filesets.FindByName('Fileset 1')
        
            # Set cal file
            Fileset.SetCalibrationFile(cal_file)
            
            for sub_ind in range(0,n_files):
                Fileset.DataFiles.Add(raw_file_names[ind+sub_ind])
                
            image_file_name = os.path.basename(raw_file_names[ind])[:-4]
            
            Sv_Var = EvFile.Variables.FindByName('Calibration Subset T1')

In [23]:
k = EvFile.Variables.FindByName('Calibration Subset T1').AsVariableAcoustic()


In [5]:
ind

1150